In [0]:
import numpy as np
import pandas as pd
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance

In [5]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


## Wczytywanie danych

In [7]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

## Dummy Model

In [10]:
feats = ['car_id']
X = df[feats].values
y = df['price_value'].values

model = DummyRegressor()
model.fit(X, y)
y_pred = model.predict(X)

mae(y, y_pred)

39465.934630440985

In [11]:
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [12]:
df = df[df['price_currency'] != 'EUR']
df.shape

(106290, 155)

## Features

In [0]:
SUFFIX_CUT = '__cut'
for feat in df.columns:
  if isinstance(df[feat][0], list):
    continue
  
  factorized_value = df[feat].factorize()[0]
  if SUFFIX_CUT in feat:
    df[feat] = factorized_value
  else:
    df[feat + SUFFIX_CUT] = factorized_value

In [15]:
cat_feats = [x for x in df.columns if SUFFIX_CUT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [16]:
X = df[cat_feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

-19566.588937368328

In [18]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(X, y)

imp = PermutationImportance(m, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

Weight,Feature
0.2533 ± 0.0040,param_napęd__cut
0.2008 ± 0.0047,param_faktura-vat__cut
0.1943 ± 0.0088,param_stan__cut
0.1423 ± 0.0086,param_rok-produkcji__cut
0.0629 ± 0.0047,param_moc__cut
0.0424 ± 0.0014,feature_kamera-cofania__cut
0.0412 ± 0.0008,param_skrzynia-biegów__cut
0.0286 ± 0.0037,param_marka-pojazdu__cut
0.0191 ± 0.0022,param_pojemność-skokowa__cut
0.0163 ± 0.0005,feature_bluetooth__cut
